## 1.Import required Libraires

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.optim import Adam
from PIL import Image

## Prepare the data

In [2]:
# Download and load the train data
train = datasets.MNIST(root = 'data', download = True, train = True, transform = ToTensor())
train_loader = DataLoader(train, batch_size = 64, shuffle = True)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.5MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 491kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.48MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 6.50MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



## 3. Define the Neural Network

In [3]:
class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1, 32,(3,3)),
        nn.ReLU(),
        nn.Conv2d(32, 64,(3,3)),
        nn.ReLU(),
        nn.Conv2d(64, 64,(3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*22*22, 10)
    )

  def forward(self, x):
    return self.model(x)

## 4. Instantiate Model, Loss Function, Optimizer

In [6]:
clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()

## 5.Train the Model

In [7]:
if __name__ == '__main__':
  for epoch in range(10):
    for batch in train_loader:
      X, y = batch
      X, y = X.to('cuda'), y.to('cuda')
      yhat = clf(X)
      loss = loss_fn(yhat, y)

      # Apply backprop
      opt.zero_grad()
      loss.backward()
      opt.step()

    print(f'Epoch:{epoch}, Loss: {loss.item()}')

Epoch:0, Loss: 0.036420952528715134
Epoch:1, Loss: 0.13969415426254272
Epoch:2, Loss: 0.000977637362666428
Epoch:3, Loss: 0.04475583881139755
Epoch:4, Loss: 8.852420432958752e-05
Epoch:5, Loss: 0.0002447290753480047
Epoch:6, Loss: 0.0011682994663715363
Epoch:7, Loss: 0.008167630061507225
Epoch:8, Loss: 0.0012787157902494073
Epoch:9, Loss: 4.5369491999736056e-05


## 6.Predict the Model

In [12]:
img = Image.open('img_3.jpg')
img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')
print(torch.argmax(clf(img_tensor)))

tensor(9, device='cuda:0')
